# Building a Convolutional Neural Network for Sea Ice/Water Classification

# Importing training/validation dataset
Note: Data used for this model can be transferred via ftp if required

In [1]:
import numpy as np

In [2]:
y_ice = np.zeros(7215051, dtype=np.int8)
y_water = np.ones(5534646, dtype=np.int8)
y = np.concatenate((y_ice,y_water))

X = np.empty((len(y), 20, 20, 4), dtype=np.float32)
X[:len(y_ice)] = np.load(r"E:\ScanSARzip\ice\TrainingDataS0_ice.npy")
X[len(y_ice):] = np.load(r"E:\ScanSARzip\water\TrainingDataS0_water.npy")

del y_ice, y_water

In [3]:
print(X.shape)
print(len(y))

(12749697, 20, 20, 4)
12749697


In [4]:
#Shuffles the dataset to ensure random selection of training/validation samples
#rng_state ensure that the shuffling stays the same for both arrays
rng_state = np.random.get_state()
np.random.shuffle(X)
np.random.set_state(rng_state)
np.random.shuffle(y)

np.save(r"E:\ScanSARzip\TrainingDataS0_shuffled.npy", X)
np.save(r"E:\ScanSARzip\ClassDataS0_shuffled.npy", y)

In [5]:
#Calculating normalization parameters
means = []
stds = []

for i in np.arange(X.shape[3]):
    means.append(np.mean(X[:,:,:,i]))
    stds.append(np.std(X[:,:,:,i]))

In [6]:
import pandas as pd
pd.DataFrame({'means':means, 'stds':stds}, 
             index=['HH','HV','Angle','Noise']).to_pickle(r"E:\Git\SeaIceCNN\Data\means_stds_S0_cnn_2021806.pkl")
print(means,stds)

[-18.896164, -27.194685, 34.271114, -28.044096] [5.2930045, 2.0444396, 8.304159, 0.6703767]


In [7]:
#Nomalizing dataset by mean and standard deviation to improve CNN accuracy
for i in np.arange(X.shape[3]):
    X[:,:,:,i] = (X[:,:,:,i] - means[i]) / stds[i]

# Building the CNN

In [8]:
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [9]:
# Initialising the CNN
classifierhh = Sequential()

In [10]:
#Initializing the convolution layer imput parameters
convlayers = 128 # Number of filters
convwindsize1 = 5 # Window Size (horizontal)
convwindsize2 = 5 # Window Size (vertical)
imheight = 20 # Height of input image
imwidth = 20 # Width of imput image
imbands = 4  # Number of image bands

In [11]:
# Step 1 - Convolution
classifierhh.add(Conv2D(convlayers, (convwindsize1, convwindsize1), strides =(1,1),
                        input_shape = (imwidth, imheight, imbands), activation = 'relu'))
# Adding a drop layer to randomly remove 10% of training samples to avoid overfitting of training dataset
#classifierhh.add(Dropout(rate = 0.1))

In [12]:
# Step 2 - Pooling
poolsize = 2 # Window Size to extract the maximum value from (MaxPooling)
classifierhh.add(MaxPooling2D(pool_size = (poolsize, poolsize)))

In [13]:
# Adding a second convolution/pool layer
classifierhh.add(Conv2D(convlayers, (convwindsize2, convwindsize2), strides = (1,1), activation = 'relu'))
classifierhh.add(MaxPooling2D(pool_size = (poolsize, poolsize)))

In [14]:
# Step 3 - Flattening
classifierhh.add(Flatten())

In [15]:
# Step 4 - Full connection
outdim1 = 1024 #Number of nodes in the 1st fully connected layer
classifierhh.add(Dense(units = outdim1, activation = 'relu'))
# Randomly drop 10% of the nodes during training to avoid overfitting of specific nodes
classifierhh.add(Dropout(rate = 0.1))
outdim2 = 128 # Number of nodes in the 2nd fully connected layer
classifierhh.add(Dense(units = outdim2, activation = 'relu'))
classifierhh.add(Dropout(rate = 0.1))
outdim3 = 1 # Number of nodes in 3rd fully connected layer. Since we only have a binary problem (ice/water),
            # only 1 dimension is needed
classifierhh.add(Dense(units = outdim3, activation = 'sigmoid'))

In [16]:
# Compiling the CNN
classifierhh.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
# Generating training stopage conditions to avoid overfitting
reduce_lr = ReduceLROnPlateau(monitor='accuracy', min_delta = 0.03, factor=0.7, patience=5, verbose = 1)
early_stop = EarlyStopping(monitor='val_accuracy', min_delta = 1E-4, patience = 10, verbose = 1)
checkpointer = ModelCheckpoint(filepath = r'E:\Git\SeaIceCNN\Data\SeaIceCNN_CheckPoint_20210806.h5', monitor = 'accuracy',
                               verbose=1,
                               save_best_only=True, save_weights_only = True)

In [18]:
#Train/Validation the CNN
history = classifierhh.fit(x = X, y = y, epochs = 100, callbacks =[reduce_lr, early_stop], validation_split = 0.3)

Train on 8924787 samples, validate on 3824910 samples
Epoch 1/100
8924787/8924787 [==============================] - 1618s 181us/sample - loss: 0.2751 - accuracy: 0.8723 - val_loss: 0.2558 - val_accuracy: 0.8814
Epoch 2/100
8924787/8924787 [==============================] - 1628s 182us/sample - loss: 0.2514 - accuracy: 0.8832 - val_loss: 0.2459 - val_accuracy: 0.8859
Epoch 3/100
8924787/8924787 [==============================] - 1609s 180us/sample - loss: 0.2439 - accuracy: 0.8869 - val_loss: 0.2410 - val_accuracy: 0.8886
Epoch 4/100
8924787/8924787 [==============================] - 1605s 180us/sample - loss: 0.2394 - accuracy: 0.8891 - val_loss: 0.2386 - val_accuracy: 0.8898
Epoch 5/100
8924787/8924787 [==============================] - 1597s 179us/sample - loss: 0.2361 - accuracy: 0.8907 - val_loss: 0.2340 - val_accuracy: 0.8919
Epoch 6/100
8924576/8924787 [============================>.] - ETA: 0s - loss: 0.2335 - accuracy: 0.8921
Epoch 00006: ReduceLROnPlateau reducing learning ra

In [19]:
#Save the CNN model to reload in the future and classify independent SAR scenes
classifierhh.save(r'E:\Git\SeaIceCNN\Data\SeaIceCNN_20210806.h5')

In [21]:
#Save the history results of the training/validation
import pickle
with open(r'E:\Git\SeaIceCNN\Data\SeaIceCNN_20210806.pkl','wb') as sfile:
    pickle.dump(history.history, sfile)

# Accuracy assessment of training validation data

In [22]:
# Predicting the classes using the trained model
Y = classifierhh.predict(X)[:,0]

In [23]:
# Determining the index split
i = int(0.8*len(y))
# Training accuracy for ice
print('Ice[training]:')
print(np.around(np.sum(np.abs(Y[:i][y[:i]==0]-y[:i][y[:i]==0])<=0.5)/(len(y[:i][y[:i]==0])),decimals = 3))
# Validation accuracy for ice
print('Ice[validation]:')
print(np.around(np.sum(np.abs(Y[i:][y[i:]==0]-y[i:][y[i:]==0])<=0.5)/(len(y[i:][y[i:]==0])),decimals = 3))
# Validation accuracy for water
print('Water[validation]:')
print(np.around(np.sum(np.abs(Y[i:][y[i:]==1]-y[i:][y[i:]==1])<=0.5)/(len(y[i:][y[i:]==1])),decimals = 3))
# Training accuracy for water
print('Water[training]:')
print(np.around(np.sum(np.abs(Y[:i][y[:i]==1]-y[:i][y[:i]==1])<=0.5)/(len(y[:i][y[:i]==1])),decimals = 3))
# Total training accuracy
print('Overall[training]:')
print(np.around(np.sum(np.abs(Y[:i]-y[:i])<=0.5)/(len(y[:i])),decimals = 3))
# Total validation accuracy
print('Overall[validation]:')
print(np.around(np.sum(np.abs(Y[i:]-y[i:])<=0.5)/(len(y[i:])),decimals = 3))
# Total accuracy for ice
print('Ice[all]:')
print(np.around(np.sum(np.abs(Y[y==0]-y[y==0])<=0.5)/len(y[y==0]),decimals = 3))
# Total accuracy for water
print('Water[all]:')
print(np.around(np.sum(np.abs(Y[y==1]-y[y==1])<=0.5)/len(y[y==1]),decimals = 3))
# Total accuracy for both classes
print('Overall[all]:')
print(np.around(np.sum(np.abs(Y-y)<=0.5)/len(y),decimals = 3))

Ice[training]:
0.938
Ice[validation]:
0.927
Water[validation]:
0.865
Water[training]:
0.878
Overall[training]:
0.912
Overall[validation]:
0.9
Ice[all]:
0.936
Water[all]:
0.876
Overall[all]:
0.909
